In [3]:
!pip install -r 'requirements.txt'

In [2]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader('cv.pdf')


In [3]:
pdf_doc = pdf_loader.load()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 57 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)
Ignoring wrong pointing object 71 0 (offset 0)


### Text Splitting

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(separators="\n",chunk_size = 300, chunk_overlap = 100)
documents = text_splitter.split_documents(pdf_doc)


In [5]:
documents

[Document(metadata={'source': 'cv.pdf', 'page': 0}, page_content="Mohammad Tanweer Salah     tanweer.salah@gmail.com | +49 17646675017  Berlin, Germany LinkedIn | GitHub  SUMMARY  Full Stack Developer with Expertise in Python, Java, .NET Core, and Generative AI   With over 5 years of practical experience and 3.5 years of professional expertise, I specialize in developing scalable web applications using Java and .NET Core. My recent projects include multiple innovative applications of Generative AI, leveraging LangChain and open-source Large Language Models (LLMs) to address diverse use cases.    SKILLS  Generative AI: Open Source LLMs, OpenAI LLM, LangChain, LangChain Agents, LangChain Tool, LLamaIndex, RAG, Vector Database, GraphDB Machine Learning: TensorFlow, TensorFlow Lite Back-End: Python, FastAPI, Java Spring Boot, .Net Core, C#, Java, Rest API, JSON Front-End: JavaScript, Typescript, Vue.js, React.js, Streamlit, HTML5, CSS3, Adobe XD, Figma Database: GraphDB, PostgreSQL , SQL, 

In [6]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.embeddings import OpenAIEmbeddings

import cassio

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 12


True

In [8]:
cassio.init(token=os.getenv('ASTRA_DB_APPLICATION_TOKEN'), database_id= os.getenv('ASTRA_DB_ID'))

In [10]:
embedding = OpenAIEmbeddings()

/Users/tanweersalah/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
astra_vector_store = Cassandra(embedding=embedding,
                               table_name='cv_test')

In [13]:
astra_vector_store.add_documents(documents)

['49f53df98a0b40cab7f079df8d4c02af',
 'a877ba3a74dd4a048061adcd11daa585',
 '3eaa6630912a4de2b610cd45e42ce515',
 '3274d9c76fd1424ca59c33c1f9d71c61']

In [14]:
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [15]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192")

In [16]:
astra_vector_index.query('what is the email ?', llm=llm)

'The email is tanweer.salah@gmail.com.'

In [17]:
astra_vector_index.query('what are skill related to gen ai', llm=llm)

'Based on the provided context, the skills related to Generative AI (Gen AI) are:\n\n* Open Source LLMs (Large Language Models)\n* OpenAI LLM\n* LangChain\n* LangChain Agents\n* LangChain Tool\n* LLamaIndex\n* RAG (likely referring to Retrieval-Augmented Generation)\n* Vector Database\n* GraphDB\n\nThese skills are mentioned in the "SKILLS" section under "Generative AI".'

In [18]:
astra_vector_index.query('is he qualified for intership position for Gen AI', llm=llm)

"Based on the provided context, it appears that Mohammad Tanweer Salah has a strong background in Generative AI, with skills and experience in:\n\n* Generative AI: Open Source LLMs, OpenAI LLM, LangChain, LangChain Agents, LangChain Tool, LLamaIndex, RAG, Vector Database, GraphDB\n* Machine Learning: TensorFlow, TensorFlow Lite\n* Projects: TanGPT (ChatGPT clone), YouTube Video and Webpage Summarizer, TanSeo (SEO optimizer App), Translator App, and more\n\nHe also has a Machine Learning Specialization certification (05/2023) and Artificial Intelligence Fundamentals AI - 900 - Microsoft Certification (06/2021).\n\nConsidering this, it seems likely that he is qualified for an internship position related to Generative AI. However, the specific requirements of the internship and the company's evaluation criteria would ultimately determine his suitability for the position."

In [21]:
# Function to drop the table
def drop_table(table_name):
    # Create a session to execute CQL commands
    session = cassio.client.cluster.connect()
    
    # Construct and execute the drop table query
    drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
    session.execute(drop_table_query)
    
    print(f"Table {table_name} has been deleted.")

In [12]:

def truncate_db(table_name):
    s = cassio.config.resolve_session()
    keyspace_name = 'default_keyspace'
    table_name = astra_vector_store.table_name
    truncate_table_query = f"DROP TABLE IF EXISTS {keyspace_name}.{table_name}"
    s.execute(truncate_table_query)

truncate_db('cv_test')